In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
from scipy.stats import norm
import math
import sympy as sym

H, D, E, t = sym.symbols('H D E t')


In [ ]:
LSF = 

column_params = {
    #'x' : [mean, std_dev]
    'H': np.array([10, 0.02]), #m
    'E': np.array([190e9, 10e9]), #N/m2
    'D': np.array([0.125, 0.0025]), #m
    't': np.array([0.005, 0.0006]), #m 
    
    'K_factor': 2,
    'P_max': 13 
}

subs = {
    'H': column_params['H'][0],
    'E': column_params['E'][0],
    'D': column_params['D'][0],
    't': column_params['t'][0]
}




3.834951969714103e-06
